In [126]:
import os
import json
import mindspore

os.environ['HTTP_PROXY'] = 'http://172.26.1.159:1090'
os.environ['HTTPS_PROXY'] = 'http://172.26.1.159:1090'

os.environ["CUDA_VISIBLE_DEVICES"] = "7"

In [127]:
def preprocess(text):
  text = text.replace("\n", "\\n").replace("\t", "\\t")
  return text

def postprocess(text):
  return text.replace("\\n", "\n").replace("\\t", "\t").replace('%20','  ')

In [128]:
text = "美国的首都是哪座城市"
text = f"用户：{text}\n小元： "
text = text.strip()
text = preprocess(text)

In [129]:
path = r"/home/geaming/.cache/huggingface/hub/models--ClueAI--ChatYuan-large-v2/snapshots/f566922340539c4c6a8e0dd86c5472155f07d367/spiece.model"

In [130]:
import sentencepiece as spm
import numpy as np
from typing import Union, List

class ChatYuanTokenizer():

    def __init__(self, path) -> None:
        self._tokenizer = spm.SentencePieceProcessor()
        self._tokenizer.Load(path)

    def _tokenize(self, text_input):
        """
        Returns a tokenized string.
        """
        tokens_ids = self._tokenizer.EncodeAsIds(text_input)
        tokens_ids = mindspore.Tensor(tokens_ids, dtype=mindspore.int64)
        tokens_ids = tokens_ids.unsqueeze(dim=0)
        # return ids
        return tokens_ids

    def _decode(self, tokens_ids):
        return self._tokenizer.DecodeIds(tokens_ids)

In [131]:
tokenizer = ChatYuanTokenizer(path)

In [132]:
# encoding对齐
print("encoding对齐".center(50, "="))
encoding = tokenizer._tokenize(text_input=text)
print(encoding)

====================encoding对齐====================
[[   12   623     5 11026  4627    15  1843   939   399     7    51   158
      5]]


In [133]:
# model对齐
print("model对齐".center(50, "="))
from mindnlp.models import T5Config, T5ForConditionalGeneration

config_path = r"/home/geaming/.cache/huggingface/hub/models--ClueAI--ChatYuan-large-v2/snapshots/f566922340539c4c6a8e0dd86c5472155f07d367/config.json"


with open(config_path, encoding='utf-8') as config:
    config = json.load(config)

ms_config = T5Config(**config)
ms_model = T5ForConditionalGeneration(ms_config)

# params
ms_dict = mindspore.load_checkpoint('/home/geaming/.cache/huggingface/hub/models--ClueAI--ChatYuan-large-v2/snapshots/f566922340539c4c6a8e0dd86c5472155f07d367/chatyuan_model.ckpt')
param_not_load = mindspore.load_param_into_net(ms_model, ms_dict)

outputs = ms_model.generate(encoding, output_scores=False, max_new_tokens=1024, num_beams=1, length_penalty=0.6)
print(outputs[0])

=====================model对齐======================


[    0 14958    10  4627    15 11646 14376     6     1]


In [134]:
# decoding对齐
print("decoding对齐".center(50, "="))
print(tokenizer._decode(outputs[0].asnumpy().tolist()))

====================decoding对齐====================
美国的首都是华盛顿特区。


In [135]:
# ckpt转换
# import logging
# def torch_to_mindspore(pth_file, size:str=None):
#     try:
#         import torch
#     except:
#         raise ImportError(f"'import torch' failed, please install torch by "
#                           f"`pip install torch` or instructions from 'https://pytorch.org'")

#     size = "mindspore" if not size else size # rename ckpt

#     from mindspore import Tensor
#     from mindspore.train.serialization import save_checkpoint

#     logging.info('Starting checkpoint conversion.')
#     ms_ckpt = []
#     state_dict = torch.load(pth_file, map_location=torch.device('cpu'))

#     for k, v in state_dict.items():
#         if 'shared.weight' in k:
#             k = k.replace('shared.weight', 'decoder.embed_tokens.embedding_table')
#         if 'relative_attention_bias.weight' in k:
#             k = k.replace('relative_attention_bias.weight', 'relative_attention_bias.embedding_table')
#         ms_ckpt.append({'name': k, 'data': Tensor(v.numpy())})

#     ms_ckpt_path = pth_file.replace('.bin','.ckpt')
#     ms_ckpt_path = ms_ckpt_path.replace('pytorch',size)
#     try:
#         save_checkpoint(ms_ckpt, ms_ckpt_path)
#     except:
#         raise RuntimeError(f'Save checkpoint to {ms_ckpt_path} failed, please checkout the path.')

#     return ms_ckpt_path

# torch_to_mindspore("/home/geaming/.cache/huggingface/hub/models--ClueAI--ChatYuan-large-v2/snapshots/f566922340539c4c6a8e0dd86c5472155f07d367/pytorch_model.bin", "chatyuan")

